# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [2]:
#jupyter nbextension enable --py --sys-prefix gmaps

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
clean_city_data = pd.read_csv("output_data/cities.csv")
clean_city_data.head()

,City_ID,Date,City,Country,Latitude,Longitude,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,0,1596594952,fortuna,US,40.60,-124.16,61.00,89,98,1.99
1,1,1596594957,sao filipe,CV,14.90,-24.50,75.97,80,61,5.30
2,2,1596594957,rikitea,PF,-23.12,-134.97,73.15,90,92,12.26
3,3,1596594835,meulaboh,ID,4.14,96.13,82.45,66,100,3.38
4,4,1596594957,port alfred,ZA,-33.59,26.89,54.68,74,98,5.79


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [8]:
# Heatmap of humidity
locations = clean_city_data[["Latitude","Longitude"]]
humidity = clean_city_data["Humidity (%)"]
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [9]:
# Narrow down cities that fit criteria and drop any results with null values
refined_city_df = clean_city_data.loc[(clean_city_data["Max Temperature (F)"] < 80) & (clean_city_data["Max Temperature (F)"] > 70) \
                                    & (clean_city_data["Humidity (%)"] < 55) \
                                    & (clean_city_data["Wind Speed (mph)"] < 12) \
                                    & (clean_city_data["Cloudiness (%)"] == 0)].dropna()
refined_city_df

,City_ID,Date,City,Country,Latitude,Longitude,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
26,26,1596594963,mandalgovi,MN,45.76,106.27,76.39,35,0,11.45
57,57,1596594971,farah,AF,32.50,62.50,77.29,18,0,7.47
128,128,1596594991,elat,IL,29.56,34.95,74.98,51,0,5.79
191,191,1596594785,aykhal,RU,66.00,111.50,75.20,31,0,4.47
250,250,1596595023,nortelandia,BR,-14.45,-56.80,72.01,39,0,4.09
290,290,1596594876,alta floresta,BR,-9.88,-56.09,73.49,48,0,4.79
366,366,1596595052,baruun-urt,MN,46.68,113.28,72.82,47,0,3.27
373,373,1596594914,presidencia roque saenz pena,AR,-26.79,-60.44,73.47,47,0,10.20
426,426,1596595068,ust-maya,RU,60.42,134.53,79.86,28,0,4.09
544,544,1596595096,khash,IR,28.22,61.22,75.65,20,0,5.77


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
# Create DataFrame called hotel_df to store hotel names along with city, country and coordinates
hotel_df = refined_city_df[["City", "Country", "Latitude", "Longitude"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
26,mandalgovi,MN,45.76,106.27,
57,farah,AF,32.50,62.50,
128,elat,IL,29.56,34.95,
191,aykhal,RU,66.00,111.50,
250,nortelandia,BR,-14.45,-56.80,
290,alta floresta,BR,-9.88,-56.09,
366,baruun-urt,MN,46.68,113.28,
373,presidencia roque saenz pena,AR,-26.79,-60.44,
426,ust-maya,RU,60.42,134.53,
544,khash,IR,28.22,61.22,


In [11]:
# Set parameters to search for a hotel
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Iterate through 
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]
    
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

hotel_df

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


,City,Country,Latitude,Longitude,Hotel Name
26,mandalgovi,MN,45.76,106.27,В.Ганбат
57,farah,AF,32.50,62.50,
128,elat,IL,29.56,34.95,Club Hotel Eilat
191,aykhal,RU,66.00,111.50,
250,nortelandia,BR,-14.45,-56.80,Hotel Karysmã
290,alta floresta,BR,-9.88,-56.09,Floresta Amazonica Hotel
366,baruun-urt,MN,46.68,113.28,Ахмадын амралт сувилал
373,presidencia roque saenz pena,AR,-26.79,-60.44,Hotel Aconcagua
426,ust-maya,RU,60.42,134.53,
544,khash,IR,28.22,61.22,دفتر پیشخوان دولت جمال زهی (خالد)


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [13]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))